# 📊 Tutorial 1: Introdução ao Time Series Forecasting Engine

Este notebook fornece uma introdução prática ao uso do Time Series Forecasting Engine, cobrindo:
1. Carregamento e visualização de dados
2. Pré-processamento básico
3. Treinamento de modelos simples
4. Avaliação e visualização de resultados

## Instalação

Certifique-se de ter o pacote instalado:
```bash
pip install -e .
```

In [ ]:
# Imports necessários
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Imports realizados com sucesso!")

## 1. Gerando Dados de Exemplo

Vamos criar uma série temporal sintética com:
- Tendência linear
- Sazonalidade anual e semanal
- Ruído aleatório

In [ ]:
def generate_sample_data(n_points=365*2, start_date='2020-01-01'):
    """
    Gera dados de série temporal sintética.
    
    Parâmetros:
    - n_points: número de pontos (dias)
    - start_date: data inicial
    """
    dates = pd.date_range(start=start_date, periods=n_points, freq='D')
    
    # Componentes
    trend = np.linspace(100, 200, n_points)  # Tendência crescente
    seasonal_yearly = 20 * np.sin(2 * np.pi * np.arange(n_points) / 365)  # Sazonalidade anual
    seasonal_weekly = 10 * np.sin(2 * np.pi * np.arange(n_points) / 7)   # Sazonalidade semanal
    noise = np.random.normal(0, 5, n_points)  # Ruído
    
    # Combinação
    values = trend + seasonal_yearly + seasonal_weekly + noise
    
    return pd.Series(values, index=dates, name='vendas')

# Gerar dados
data = generate_sample_data()

print(f"Dados gerados: {len(data)} observações")
print(f"Período: {data.index[0]} a {data.index[-1]}")
print(f"\nPrimeiras 5 observações:")
print(data.head())

## 2. Visualização Inicial

In [ ]:
# Importar o visualizador
import sys
sys.path.insert(0, '../')
from src.visualization import TimeSeriesVisualizer

visualizer = TimeSeriesVisualizer()

# Plotar série temporal
fig = visualizer.plot_time_series(
    data,
    title="Série Temporal de Vendas",
    show_trend=True
)
plt.show()

print("\nEstatísticas descritivas:")
print(data.describe())

## 3. Divisão Treino/Teste

Para séries temporais, sempre dividimos os dados respeitando a ordem temporal.

In [ ]:
# Divisão 80/20
train_size = int(len(data) * 0.8)
train = data[:train_size]
test = data[train_size:]

print(f"Conjunto de treino: {len(train)} observações ({train.index[0]} a {train.index[-1]})")
print(f"Conjunto de teste: {len(test)} observações ({test.index[0]} a {test.index[-1]})")

# Visualizar divisão
plt.figure(figsize=(14, 6))
plt.plot(train.index, train.values, label='Treino', color='blue', alpha=0.7)
plt.plot(test.index, test.values, label='Teste', color='green', alpha=0.7)
plt.axvline(x=train.index[-1], color='red', linestyle='--', label='Divisão')
plt.xlabel('Data')
plt.ylabel('Vendas')
plt.title('Divisão Treino/Teste')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Pré-processamento Básico

In [ ]:
from src.preprocessing import TimeSeriesPreprocessor

preprocessor = TimeSeriesPreprocessor()

# Verificar valores faltantes
print(f"Valores faltantes: {train.isna().sum()}")

# Detectar outliers
outliers_mask = preprocessor.detect_outliers(train, method='iqr', threshold=3.0)
print(f"Outliers detectados: {outliers_mask.sum()}")

# Remover outliers
train_clean = preprocessor.remove_outliers(train, method='iqr', threshold=3.0)
print(f"Dados após remoção de outliers: {len(train_clean)} observações")

# Testar estacionariedade
is_stationary, test_result = preprocessor.test_stationarity(train_clean)
print(f"\nTeste de Estacionariedade (ADF):")
print(f"  Estacionário: {is_stationary}")
print(f"  Estatística ADF: {test_result['adf_statistic']:.4f}")
print(f"  p-valor: {test_result['p_value']:.4f}")

## 5. Modelo ARIMA

Vamos treinar um modelo ARIMA com seleção automática de parâmetros.

In [ ]:
from src.models import ARIMAForecaster

# Criar e treinar modelo
arima = ARIMAForecaster(auto_select=True)
print("Treinando modelo ARIMA...")
arima.fit(train_clean)
print("✓ Modelo treinado!")

# Fazer previsões
predictions = arima.predict(steps=len(test))
print(f"\nPrevisões geradas: {len(predictions)} pontos")

# Obter intervalos de confiança
pred_with_intervals = arima.predict_with_intervals(steps=len(test), confidence=0.95)
predictions, lower_bound, upper_bound = pred_with_intervals
print("✓ Intervalos de confiança calculados")

## 6. Avaliação do Modelo

In [ ]:
from src.evaluation import ModelEvaluator

evaluator = ModelEvaluator()

# Calcular métricas
metrics = evaluator.calculate_metrics(test.values, predictions.values)

print("Métricas de Erro:")
print(f"  MAE (Erro Absoluto Médio):          {metrics['MAE']:.2f}")
print(f"  RMSE (Raiz do Erro Quadrático):     {metrics['RMSE']:.2f}")
print(f"  MAPE (Erro Percentual Absoluto):    {metrics['MAPE']:.2f}%")
print(f"  sMAPE (MAPE Simétrico):             {metrics['sMAPE']:.2f}%")
print(f"  R² (Coeficiente de Determinação):   {metrics['R2']:.4f}")
print(f"  MASE (Erro Absoluto Escalado):      {metrics['MASE']:.4f}")

## 7. Visualização dos Resultados

In [ ]:
# Plot da previsão
fig = visualizer.plot_forecast(
    train_clean,
    test,
    predictions,
    lower_bound,
    upper_bound,
    title="ARIMA: Previsão vs Real"
)
plt.show()

In [ ]:
# Análise de resíduos
if hasattr(arima, 'residuals_') and arima.residuals_ is not None:
    fig = visualizer.plot_residuals(arima.residuals_)
    plt.show()
    
    # Estatísticas dos resíduos
    residual_stats = evaluator.residual_analysis(arima, test, predictions)
    print("\nAnálise de Resíduos:")
    print(f"  Média: {residual_stats['mean']:.4f}")
    print(f"  Desvio padrão: {residual_stats['std']:.4f}")
    print(f"  Normalmente distribuído: {residual_stats['is_normal']}")
    print(f"  Ruído branco (sem autocorrelação): {residual_stats['is_white_noise']}")

## 8. Modelo Prophet

Agora vamos testar o Prophet, que é especialmente bom para capturar múltiplas sazonalidades.

In [ ]:
from src.models import ProphetForecaster

# Criar e treinar modelo
prophet = ProphetForecaster(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)

print("Treinando modelo Prophet...")
prophet.fit(train_clean)
print("✓ Modelo treinado!")

# Fazer previsões
prophet_predictions = prophet.predict(steps=len(test))
prophet_pred_intervals = prophet.predict_with_intervals(steps=len(test), confidence=0.95)
prophet_predictions, prophet_lower, prophet_upper = prophet_pred_intervals

# Avaliar
prophet_metrics = evaluator.calculate_metrics(test.values, prophet_predictions.values)

print("\nMétricas Prophet:")
print(f"  MAE:   {prophet_metrics['MAE']:.2f}")
print(f"  RMSE:  {prophet_metrics['RMSE']:.2f}")
print(f"  MAPE:  {prophet_metrics['MAPE']:.2f}%")

In [ ]:
# Visualizar Prophet
fig = visualizer.plot_forecast(
    train_clean,
    test,
    prophet_predictions,
    prophet_lower,
    prophet_upper,
    title="Prophet: Previsão vs Real"
)
plt.show()

## 9. Comparação de Modelos

In [ ]:
# Comparar modelos
models = [arima, prophet]
comparison = evaluator.compare_models(models, train_clean, test)

print("\nComparação de Modelos:")
print(comparison)

# Visualizar comparação
fig = visualizer.plot_model_comparison(comparison, metric='RMSE')
plt.show()

## 10. Cross-Validation

Validação cruzada para séries temporais.

In [ ]:
# Criar novo modelo para CV
cv_model = ARIMAForecaster(auto_select=True)

# Realizar cross-validation
print("Executando cross-validation (pode demorar alguns minutos)...")
cv_results = evaluator.time_series_cv(
    model=cv_model,
    data=data,
    n_splits=5,
    test_size=30
)

print("\nResultados de Cross-Validation:")
print(f"  RMSE: {cv_results['RMSE_mean']:.2f} ± {cv_results['RMSE_std']:.2f}")
print(f"  MAE:  {cv_results['MAE_mean']:.2f} ± {cv_results['MAE_std']:.2f}")
print(f"  MAPE: {cv_results['MAPE_mean']:.2f}% ± {cv_results['MAPE_std']:.2f}%")

## Conclusão

Neste tutorial, você aprendeu:

1. ✅ Como gerar e visualizar dados de séries temporais
2. ✅ Como realizar pré-processamento básico
3. ✅ Como treinar modelos ARIMA e Prophet
4. ✅ Como avaliar modelos usando múltiplas métricas
5. ✅ Como comparar diferentes modelos
6. ✅ Como usar cross-validation para séries temporais

### Próximos Passos

- **Tutorial 2:** Deep Learning com LSTM
- **Tutorial 3:** Modelos Ensemble
- **Tutorial 4:** Pré-processamento Avançado
- **Tutorial 5:** Casos de Uso Reais